In [1]:
# Cell 0: The Definitive Installation Command

# This command explicitly targets and forcefully upgrades all core LangChain components,
# including the problematic 'langchain-community' package. This ensures that all
# related packages are brought to their latest, mutually compatible versions,
# resolving the final dependency conflict.

#!pip install -q --upgrade \
#    langchain \
#    langchain-core \
#    langchain-community \
#    langgraph \
#    langsmith \
#    langchain-openai \
#    dateparser \
#    pytz \
##    google-api-python-client \
 #   google-auth-oauthlib \
 #   requests

#print("\n✅ All dependencies have been forcefully upgraded to their latest stable versions.")
#print("   This should resolve all dependency conflicts.")
#print("‼️ IMPORTANT: Please restart the Jupyter kernel now before proceeding to the next cell.")
#print("   (Go to the 'Kernel' menu -> 'Restart Kernel...')")

In [1]:
# Cell 1: Imports (Corrected)
# --- Core Libraries ---
import json
import os
from datetime import datetime, timedelta
from threading import Thread
import logging

# --- Web Server ---
from flask import Flask, request, jsonify

# --- Data Validation & Time Handling ---
from pydantic import BaseModel, Field
from typing import List, Dict, Optional, Any
import pytz
from dateutil.parser import parse as date_parse

# --- Google Calendar API ---
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# --- LangChain & LangGraph for Agentic Workflow ---
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
# The problematic line that caused the warning has been removed.

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# --- Suppress unnecessary warnings ---
logging.basicConfig(level=logging.INFO)
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

print("Cell 1/13: All libraries imported successfully (Warning resolved).")

Cell 1/13: All libraries imported successfully (Warning resolved).


In [2]:
# Cell 2: LLM and Environment Configuration (Final Path Correction)

# --- vLLM Server Details for Llama 3.1 ---
# We are now pointing to the Llama 3.1 server on port 4000.
BASE_URL = "http://localhost:4000/v1"

# CRITICAL FIX: The model path MUST match the path used to start the vLLM server.
# The server was started with the RELATIVE path, not the absolute /home/user path.
MODEL_PATH = "/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct"

API_KEY = "not-needed"

# --- Initialize the LLM client for LangChain ---
llm = ChatOpenAI(
    model=MODEL_PATH,
    api_key=API_KEY,
    base_url=BASE_URL,
    max_tokens=500,
)

# --- Define working hours and other constants ---
WORK_DAY_START_HOUR = 9
WORK_DAY_END_HOUR = 18

print("Cell 2/13: LLM client configured for Llama 3.1 with CORRECT model path.")

Cell 2/13: LLM client configured for Llama 3.1 with CORRECT model path.


In [3]:
# Cell 3: Pydantic State and Data Models (Upgraded for Date Ranges)

class ExtractedInfo(BaseModel):
    """Holds the structured information, now capable of understanding date ranges."""
    meeting_summary: str = Field(description="A brief summary or title for the meeting.")
    duration_minutes: int = Field(description="The duration of the meeting in minutes.")
    # The LLM will now extract a start and end date for the search window
    search_start_date: str = Field(description="The start date for the search window, in YYYY-MM-DD format.")
    search_end_date: str = Field(description="The end date for the search window, in YYYY-MM-DD format.")
    time_preference: Optional[str] = Field(default=None, description="Any specific time preference like 'afternoon', 'morning', or 'at 2 PM'.")

class Attendee(BaseModel):
    """Represents a single attendee and their status, with timezone info."""
    email: str
    timezone: str = "UTC"
    events: List[Dict[str, Any]] = Field(default_factory=list)

class ProposedSlot(BaseModel):
    """Represents a concrete time slot for the meeting."""
    start: datetime
    end: datetime

class AgentState(BaseModel):
    """The main state object for our LangGraph agent."""
    request_data: Dict[str, Any]
    attendees: Optional[List[Attendee]] = Field(default_factory=list)
    organizer: Optional[str] = None
    all_attendee_emails: Optional[List[str]] = Field(default_factory=list)
    extracted_info: Optional[ExtractedInfo] = None
    proposed_slot: Optional[ProposedSlot] = None
    status_message: str = "Processing..."
    final_output: Optional[Dict[str, Any]] = None
    conflict_resolution: Optional[Dict[str, Any]] = None  # NEW: Store conflict resolution data

    class Config:
        arbitrary_types_allowed = True

print("Cell 3/13: Pydantic models UPGRADED for multi-day scheduling.")

Cell 3/13: Pydantic models UPGRADED for multi-day scheduling.


In [4]:
# Cell 4: Agent Tools (Final Version with Docstrings Restored)

# --- SIMULATED USER TIMEZONE DATABASE ---
USER_TIMEZONES = {
    "userone.amd@gmail.com": "Asia/Kolkata",
    "usertwo.amd@gmail.com": "Asia/Kolkata",
    "userthree.amd@gmail.com": "Asia/Kolkata"
}
# ---

@tool
def extract_meeting_details_from_email(email_content: str, request_datetime_str: str) -> ExtractedInfo:
    """
    [UPGRADED NLU]
    Uses the LLM to parse email content, now understanding date ranges.
    """
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", """You are an expert assistant for parsing meeting requests. Your task is to extract key details into a structured JSON format.
The current date for context is {request_datetime}.
- If the user specifies a range (e.g., "next week", "between Monday and Wednesday"), provide a start and end date.
- If the user specifies a single day (e.g., "tomorrow", "on Tuesday"), the start and end date will be the same.
- A "week" starts on Monday and ends on Friday.
- "time_preference" can be 'morning', 'afternoon', 'evening', or a specific time like '2 PM'.

You MUST return ONLY a valid JSON object string. Do not add any other text.

Example 1:
User request: "Let's meet sometime next week for an hour to discuss the project."
Current Date: 2025-07-18 (Friday)
Output:
{{
  "meeting_summary": "discuss the project",
  "duration_minutes": 60,
  "search_start_date": "2025-07-21",
  "search_end_date": "2025-07-25",
  "time_preference": null
}}

Example 2:
User request: "Can we find 30 minutes on Monday afternoon?"
Current Date: 2025-07-18 (Friday)
Output:
{{
  "meeting_summary": "find 30 minutes",
  "duration_minutes": 30,
  "search_start_date": "2025-07-21",
  "search_end_date": "2025-07-21",
  "time_preference": "afternoon"
}}"""),
        ("human", "Here is the email content:\n\n---\n{email_content}\n---")
    ])
    
    chain = prompt_template | llm
    
    try:
        response = chain.invoke({"email_content": email_content, "request_datetime": date_parse(request_datetime_str).strftime('%Y-%m-%d')})
        response_content = response.content
        json_start = response_content.find('{')
        json_end = response_content.rfind('}')
        if json_start != -1 and json_end != -1:
            return ExtractedInfo(**json.loads(response_content[json_start:json_end+1]))
        else:
            raise ValueError("No JSON object found in the LLM response.")
    except Exception as e:
        logging.error(f"LLM parsing failed! Error: {e}")
        raise ValueError("LLM failed to return a valid JSON object.")

@tool
def get_calendar_events(attendee_email: str, start_time: datetime, end_time: datetime) -> List[Dict[str, Any]]:
    """
    [CRITICAL FIX]
    Fetches events from a user's Google Calendar for a given time window.
    It now attempts to extract attendee information for each event gracefully.
    """
    events_list = []; token_path = f"Keys/{attendee_email.split('@')[0]}.token"
    if not os.path.exists(token_path): return []
    try:
        creds = Credentials.from_authorized_user_file(token_path)
        service = build("calendar", "v3", credentials=creds)
        events_result = service.events().list(calendarId='primary', timeMin=start_time.isoformat(), timeMax=end_time.isoformat(), singleEvents=True, orderBy='startTime').execute()
        for event in events_result.get('items', []):
            start = date_parse(event['start'].get('dateTime', event['start'].get('date'))); end = date_parse(event['end'].get('dateTime', event['end'].get('date')))
            try:
                event_attendees = event.get('attendees', [])
                if event_attendees: attendee_list = [att['email'] for att in event_attendees]; num_attendees = len(attendee_list)
                else: num_attendees = 1; attendee_list = [attendee_email]
            except KeyError: num_attendees = 1; attendee_list = [attendee_email]
            events_list.append({"summary": event.get("summary", "No Title"), "start": start, "end": end, "NumAttendees": num_attendees, "Attendees": attendee_list})
        return events_list
    except Exception as e: logging.error(f"Error fetching calendar for {attendee_email}: {e}"); return []

@tool
def find_available_slots(attendees: List[Attendee], busy_slots: List[Dict[str, Any]], search_start_utc: datetime, duration_minutes: int) -> List[ProposedSlot]:
    """
    [CRITICAL FIX]
    Finds common available slots within the overlapping working hours of all attendees.
    """
    utc_tz = pytz.UTC; golden_start, golden_end = utc_tz.localize(datetime.min), utc_tz.localize(datetime.max)
    for attendee in attendees:
        local_tz = pytz.timezone(attendee.timezone); local_workday_start = search_start_utc.astimezone(local_tz).replace(hour=WORK_DAY_START_HOUR, minute=0, second=0, microsecond=0); local_workday_end = search_start_utc.astimezone(local_tz).replace(hour=WORK_DAY_END_HOUR, minute=0, second=0, microsecond=0); utc_workday_start, utc_workday_end = local_workday_start.astimezone(utc_tz), local_workday_end.astimezone(utc_tz); golden_start, golden_end = max(golden_start, utc_workday_start), min(golden_end, utc_workday_end)
    if golden_start >= golden_end: return []
    utc_busy_intervals = sorted([(e['start'].astimezone(utc_tz), e['end'].astimezone(utc_tz)) for e in busy_slots]);
    if not utc_busy_intervals: merged = []
    else:
        merged = [utc_busy_intervals[0]];
        for cs, ce in utc_busy_intervals[1:]:
            ls, le = merged[-1];
            if cs < le: merged[-1] = (ls, max(le, ce))
            else: merged.append((cs, ce))
    free_slots = []; current_time = golden_start
    for bs, be in merged:
        if current_time < bs: free_slots.append((current_time, bs));
        current_time = max(current_time, be)
    if current_time < golden_end: free_slots.append((current_time, golden_end))
    meeting_duration = timedelta(minutes=duration_minutes); potential_slots = []
    for fs, fe in free_slots:
        ss = fs
        while ss + meeting_duration <= fe:
            potential_slots.append(ProposedSlot(start=ss, end=ss + meeting_duration)); ss += timedelta(minutes=15)
            if len(potential_slots) >= 5: break
        if len(potential_slots) >= 5: break
    return potential_slots


print("Cell 4/13: Agent tools FINALIZED with restored docstrings.")

Cell 4/13: Agent tools FINALIZED with restored docstrings.


In [5]:
# Cell 4.5: Advanced Conflict Resolution & Priority Negotiation System

from enum import Enum
from typing import Tuple

class MeetingPriority(Enum):
    LOW = "low"
    MEDIUM = "medium"  
    HIGH = "high"
    URGENT = "urgent"

class ConflictType(Enum):
    RESCHEDULABLE = "reschedulable"  # Coffee break, lunch, personal time
    NEGOTIABLE = "negotiable"        # Internal meetings, 1:1s
    IMMOVABLE = "immovable"         # Client calls, interviews, presentations

# Meeting type classification for conflict resolution
RESCHEDULABLE_KEYWORDS = [
    "break", "coffee", "lunch", "personal", "gym", "workout", "walk",
    "casual", "informal", "check-in", "catch up", "social"
]

NEGOTIABLE_KEYWORDS = [
    "1:1", "one-on-one", "team meeting", "standup", "sync", "update",
    "planning", "brainstorm", "internal", "discussion", "review"
]

IMMOVABLE_KEYWORDS = [
    "client", "customer", "interview", "presentation", "demo", "call",
    "external", "vendor", "board", "executive", "ceo", "urgent", "critical"
]

@tool
def analyze_meeting_priority(email_content: str) -> MeetingPriority:
    """
    Uses LLM to determine meeting priority based on email content and urgency indicators.
    """
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", """You are an expert at analyzing meeting priority and urgency. 
Analyze the email content and determine the priority level based on:
- Urgency words: "urgent", "asap", "immediately", "critical", "emergency"
- Business impact: client-facing, revenue impact, deadline-driven
- Language tone: formal vs casual
- Timeline pressure: "today", "tomorrow", "end of week"

Return ONLY one word from: low, medium, high, urgent

Examples:
"Let's catch up sometime this week" → low
"We need to discuss the project status" → medium  
"Client is waiting for our response by EOD" → high
"URGENT: Server is down, need immediate meeting" → urgent"""),
        ("human", "Email content: {email_content}")
    ])
    
    chain = prompt_template | llm
    
    try:
        response = chain.invoke({"email_content": email_content})
        priority_str = response.content.strip().lower()
        
        # Map to enum with fallback
        priority_mapping = {
            "low": MeetingPriority.LOW,
            "medium": MeetingPriority.MEDIUM,
            "high": MeetingPriority.HIGH,
            "urgent": MeetingPriority.URGENT
        }
        return priority_mapping.get(priority_str, MeetingPriority.MEDIUM)
    except Exception as e:
        logging.error(f"Priority analysis failed: {e}")
        return MeetingPriority.MEDIUM

@tool
def classify_conflicting_meetings(busy_events: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Classifies existing meetings based on how easily they can be rescheduled.
    """
    classified_events = []
    
    for event in busy_events:
        summary = event.get('summary', '').lower()
        
        # Determine conflict type based on meeting title
        if any(keyword in summary for keyword in RESCHEDULABLE_KEYWORDS):
            conflict_type = ConflictType.RESCHEDULABLE
        elif any(keyword in summary for keyword in NEGOTIABLE_KEYWORDS):
            conflict_type = ConflictType.NEGOTIABLE  
        elif any(keyword in summary for keyword in IMMOVABLE_KEYWORDS):
            conflict_type = ConflictType.IMMOVABLE
        else:
            # Default classification logic
            num_attendees = event.get('NumAttendees', 1)
            if num_attendees == 1:
                conflict_type = ConflictType.RESCHEDULABLE
            elif num_attendees <= 3:
                conflict_type = ConflictType.NEGOTIABLE
            else:
                conflict_type = ConflictType.IMMOVABLE
        
        classified_event = event.copy()
        classified_event['conflict_type'] = conflict_type
        classified_event['reschedulability_score'] = _calculate_reschedulability_score(event, conflict_type)
        classified_events.append(classified_event)
    
    return classified_events

def _calculate_reschedulability_score(event: Dict[str, Any], conflict_type: ConflictType) -> float:
    """
    Calculates how easy it is to reschedule this meeting (0.0 = impossible, 1.0 = very easy).
    """
    base_scores = {
        ConflictType.RESCHEDULABLE: 0.9,
        ConflictType.NEGOTIABLE: 0.6,
        ConflictType.IMMOVABLE: 0.1
    }
    
    score = base_scores[conflict_type]
    
    # Adjust based on meeting characteristics
    summary = event.get('summary', '').lower()
    num_attendees = event.get('NumAttendees', 1)
    
    # Single-person events are easier to reschedule
    if num_attendees == 1:
        score += 0.1
    
    # Reduce score for large meetings
    if num_attendees > 5:
        score -= 0.2
    
    # Specific keywords that affect reschedulability
    if any(word in summary for word in ['recurring', 'weekly', 'daily']):
        score += 0.1  # Recurring meetings are easier to skip once
    
    if any(word in summary for word in ['deadline', 'launch', 'release']):
        score -= 0.3  # Deadline-related meetings are harder to move
    
    return max(0.0, min(1.0, score))

@tool
def generate_conflict_resolution_suggestions(
    meeting_priority: MeetingPriority,
    conflicting_events: List[Dict[str, Any]],
    requested_duration: int,
    search_date: str
) -> Dict[str, Any]:
    """
    Generates intelligent suggestions for resolving scheduling conflicts.
    """
    suggestions = {
        "negotiation_strategy": "",
        "reschedulable_conflicts": [],
        "alternative_suggestions": [],
        "priority_override_recommendation": False
    }
    
    # Find reschedulable conflicts
    reschedulable = [e for e in conflicting_events if e.get('reschedulability_score', 0) >= 0.7]
    negotiable = [e for e in conflicting_events if 0.4 <= e.get('reschedulability_score', 0) < 0.7]
    
    suggestions["reschedulable_conflicts"] = reschedulable
    
    # Generate strategy based on meeting priority
    if meeting_priority in [MeetingPriority.HIGH, MeetingPriority.URGENT]:
        if reschedulable:
            suggestions["negotiation_strategy"] = f"HIGH PRIORITY: Suggest rescheduling {len(reschedulable)} flexible meeting(s) to accommodate this {meeting_priority.value} priority request."
            suggestions["priority_override_recommendation"] = True
            
        elif negotiable:
            suggestions["negotiation_strategy"] = f"URGENT REQUEST: Consider negotiating with attendees to reschedule {len(negotiable)} potentially flexible meeting(s)."
            suggestions["priority_override_recommendation"] = True
            
        else:
            suggestions["negotiation_strategy"] = "CONFLICT DETECTED: All existing meetings appear immovable. Suggest alternative time slots or shorter duration."
    
    else:  # LOW or MEDIUM priority
        if reschedulable:
            suggestions["negotiation_strategy"] = f"POLITE REQUEST: {len(reschedulable)} flexible meeting(s) could potentially be rescheduled if attendees agree."
        else:
            suggestions["negotiation_strategy"] = "RESPECTFUL APPROACH: All existing meetings appear important. Suggest alternative times or dates."
    
    # Generate alternative suggestions
    alternatives = []
    
    # Suggest earlier/later in the same day
    alternatives.append(f"Earlier time slot on {search_date} (before existing meetings)")
    alternatives.append(f"Later time slot on {search_date} (after existing meetings)")
    
    # Suggest shorter duration
    if requested_duration > 30:
        alternatives.append(f"Shorter meeting duration ({requested_duration//2} minutes instead of {requested_duration})")
    
    # Suggest next business day
    alternatives.append("Next available business day with full availability")
    
    # Suggest split meeting
    if requested_duration >= 60:
        alternatives.append(f"Split into two shorter sessions ({requested_duration//2} minutes each)")
    
    suggestions["alternative_suggestions"] = alternatives
    
    return suggestions

@tool
def generate_negotiation_message(
    meeting_summary: str,
    priority: MeetingPriority, 
    conflict_resolution: Dict[str, Any],
    organizer_email: str
) -> str:
    """
    Generates a polite but assertive negotiation message using LLM.
    """
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", """You are an expert executive assistant who excels at diplomatic negotiation. 
Generate a professional but persuasive message for scheduling conflicts.

Key principles:
- Be respectful but assertive based on priority level
- Provide clear alternatives
- Use appropriate business language
- Include specific rescheduling suggestions when available

Priority levels and tone:
- URGENT: Direct and immediate, emphasize business impact
- HIGH: Professional urgency, stress importance  
- MEDIUM: Polite but confident request
- LOW: Very diplomatic, offer maximum flexibility"""),
        ("human", """
Meeting: {meeting_summary}
Priority: {priority}
Strategy: {strategy}
Reschedulable conflicts: {reschedulable_count}
Alternatives: {alternatives}
Organizer: {organizer}

Generate a short and concise (within 2-3 lines), professional message for attendees explaining the scheduling conflict and proposing solutions.""")
    ])
    
    chain = prompt_template | llm
    
    try:
        response = chain.invoke({
            "meeting_summary": meeting_summary,
            "priority": priority.value,
            "strategy": conflict_resolution.get("negotiation_strategy", ""),
            "reschedulable_count": len(conflict_resolution.get("reschedulable_conflicts", [])),
            "alternatives": ", ".join(conflict_resolution.get("alternative_suggestions", [])[:3]),
            "organizer": organizer_email
        })
        return response.content
    except Exception as e:
        logging.error(f"Message generation failed: {e}")
        return f"Scheduling conflict detected for {meeting_summary}. Please review available alternatives and advise on preferred approach."

print("Cell 4.5/13: Advanced conflict resolution and priority negotiation system added.")

Cell 4.5/13: Advanced conflict resolution and priority negotiation system added.


In [6]:
# Cell 5: Graph Nodes (Final Version with Multi-Day Search Logic)

def get_utc_timezone(): return pytz.UTC

def initialize_state(state: AgentState) -> AgentState:
    """Node 1: Initializes the state and injects user timezones."""
    request = state.request_data
    organizer = request['From']
    attendee_emails = [att['email'] for att in request['Attendees']]
    all_emails = list(set([organizer] + attendee_emails))
    state.attendees = [Attendee(email=email, timezone=USER_TIMEZONES.get(email, "UTC")) for email in all_emails]
    state.organizer = organizer
    state.all_attendee_emails = all_emails
    state.status_message = "State initialized."
    return state

def extract_details(state: AgentState) -> AgentState:
    """Node 2: Calls the LLM to get the search range."""
    logging.info("Extracting details and date range using LLM...")
    try:
        state.extracted_info = extract_meeting_details_from_email.invoke({"email_content": state.request_data['EmailContent'], "request_datetime_str": state.request_data['Datetime']})
        state.status_message = f"Search window identified: {state.extracted_info.search_start_date} to {state.extracted_info.search_end_date}"
    except ValueError:
        state.status_message = "ERROR: LLM failed to extract details."
        state.extracted_info = None
    return state
    
def search_for_slot_in_range(state: AgentState) -> AgentState:
    """
    [ENHANCED CORE LOGIC NODE WITH CONFLICT RESOLUTION]
    Iterates through the date range and finds slots. If none found, triggers conflict resolution.
    """
    start_date = date_parse(state.extracted_info.search_start_date).date()
    end_date = date_parse(state.extracted_info.search_end_date).date()
    
    # Analyze meeting priority first
    meeting_priority = analyze_meeting_priority.invoke({"email_content": state.request_data['EmailContent']})
    
    # Store all conflicts encountered for potential resolution
    all_daily_conflicts = []
    
    current_day = start_date
    while current_day <= end_date:
        # We only check business days (Monday=0, Sunday=6)
        if current_day.weekday() < 5:
            logging.info(f"--- Searching for slots on {current_day.strftime('%Y-%m-%d')} ---")
            
            # The search must start on the current day in a neutral timezone (UTC)
            search_start_utc = get_utc_timezone().localize(datetime.combine(current_day, datetime.min.time()))
            
            # Fetch all events for this day for all attendees to be efficient
            all_events_for_day = []
            for attendee in state.attendees:
                day_start_local = search_start_utc.astimezone(pytz.timezone(attendee.timezone)).replace(hour=0, minute=0, second=0)
                day_end_local = day_start_local + timedelta(days=1)
                all_events_for_day.extend(get_calendar_events.invoke({"attendee_email": attendee.email, "start_time": day_start_local, "end_time": day_end_local}))
            
            # Call our powerful find_available_slots tool for this specific day
            available_slots = find_available_slots.invoke({
                "attendees": state.attendees,
                "busy_slots": all_events_for_day,
                "search_start_utc": search_start_utc,
                "duration_minutes": state.extracted_info.duration_minutes
            })
            
            if available_slots:
                # SUCCESS! We found a slot.
                first_slot = available_slots[0]
                state.proposed_slot = first_slot
                state.status_message = f"Success! Found an available slot on {current_day.strftime('%Y-%m-%d')}."
                logging.info(f"{state.status_message} Proposed UTC time: {first_slot.start.isoformat()}")
                # Exit the node and the loop immediately
                return state
            else:
                # Store conflicts for this day for potential resolution
                if all_events_for_day:
                    all_daily_conflicts.append({
                        "date": current_day.strftime('%Y-%m-%d'),
                        "events": all_events_for_day
                    })
        
        # Move to the next day
        current_day += timedelta(days=1)
    
    # If we reach here, no perfect slots were found
    # Trigger intelligent conflict resolution
    logging.warning("No perfect slots found. Initiating conflict resolution...")
    logging.info(f"Meeting priority detected: {meeting_priority.value}")
    logging.info(f"Daily conflicts found: {len(all_daily_conflicts)}")
    
    if all_daily_conflicts:
        # ALWAYS attempt conflict resolution when conflicts are found, regardless of priority
        # The resolution strategy will vary based on priority level
        state = _attempt_conflict_resolution(state, all_daily_conflicts, meeting_priority)
    else:
        # No conflicts found, just no available time slots
        state.status_message = "Search complete. No available slots found in the specified date range."
        state.proposed_slot = None
        # Still add basic conflict resolution info for debugging
        state.conflict_resolution = {
            "priority_level": meeting_priority.value,
            "resolution_possible": False,
            "no_conflicts_found": True,
            "alternative_suggestions": [
                "Consider scheduling for next available business day",
                "Reduce meeting duration to find smaller time slots",
                "Schedule during lunch hours if urgent",
                "Consider early morning or late evening slots"
            ]
        }
    
    return state


def _attempt_conflict_resolution(state: AgentState, daily_conflicts: List[Dict], priority: MeetingPriority) -> AgentState:
    """
    Helper function to attempt conflict resolution for ALL meetings with conflicts.
    """
    logging.info(f"Attempting conflict resolution for {priority.value} priority meeting...")
    logging.info(f"Analyzing {len(daily_conflicts)} days with conflicts...")
    
    best_resolution = None
    best_day = None
    highest_reschedulability = 0.0
    
    # Analyze each day's conflicts
    for day_data in daily_conflicts:
        events = day_data["events"]
        date_str = day_data["date"]
        
        logging.info(f"Day {date_str}: Found {len(events)} conflicting events")
        
        # Classify conflicting meetings
        classified_events = classify_conflicting_meetings.invoke({"busy_events": events})
        
        # Calculate total reschedulability for this day
        total_reschedulability = sum(e.get('reschedulability_score', 0) for e in classified_events)
        avg_reschedulability = total_reschedulability / len(classified_events) if classified_events else 0
        
        logging.info(f"Day {date_str}: Average reschedulability = {avg_reschedulability:.2f}")
        
        # If this day has better reschedulability, consider it
        if avg_reschedulability > highest_reschedulability:
            highest_reschedulability = avg_reschedulability
            best_day = date_str
            
            # Generate conflict resolution suggestions
            best_resolution = generate_conflict_resolution_suggestions.invoke({
                "meeting_priority": priority,
                "conflicting_events": classified_events,
                "requested_duration": state.extracted_info.duration_minutes,
                "search_date": date_str
            })
    
    # ALWAYS provide conflict resolution details, adjust threshold based on priority
    resolution_threshold = 0.4 if priority in [MeetingPriority.HIGH, MeetingPriority.URGENT] else 0.6
    
    if best_resolution and highest_reschedulability >= resolution_threshold:
        # Generate negotiation message
        negotiation_message = generate_negotiation_message.invoke({
            "meeting_summary": state.extracted_info.meeting_summary,
            "priority": priority,
            "conflict_resolution": best_resolution,
            "organizer_email": state.organizer
        })
        
        state.status_message = f"CONFLICT RESOLUTION: Found potential solution for {best_day} with {len(best_resolution['reschedulable_conflicts'])} reschedulable meeting(s)."
        
        # Store resolution details in metadata for the final response
        state.conflict_resolution = {
            "priority_level": priority.value,
            "recommended_date": best_day,
            "reschedulable_meetings": len(best_resolution.get('reschedulable_conflicts', [])),
            "negotiation_strategy": best_resolution.get("negotiation_strategy", ""),
            "alternative_suggestions": best_resolution.get("alternative_suggestions", []),
            "negotiation_message": negotiation_message,
            "override_recommended": best_resolution.get("priority_override_recommendation", False),
            "total_days_analyzed": len(daily_conflicts),
            "best_reschedulability_score": highest_reschedulability
        }
        
        logging.info(f"Conflict resolution prepared for {best_day} with reschedulability {highest_reschedulability:.2f}")
        
    else:
        # Still provide detailed analysis even if no good resolution found
        state.status_message = f"CONFLICT ANALYSIS: Limited reschedulability found. Priority: {priority.value}. Best score: {highest_reschedulability:.2f}"
        state.conflict_resolution = {
            "priority_level": priority.value,
            "resolution_possible": False,
            "total_days_analyzed": len(daily_conflicts),
            "best_reschedulability_score": highest_reschedulability,
            "recommended_date": best_day,
            "analysis_reason": f"Reschedulability score {highest_reschedulability:.2f} below threshold {resolution_threshold}",
            "alternative_suggestions": [
                "Consider scheduling for next available business day",
                "Reduce meeting duration to find smaller time slots", 
                "Schedule during lunch hours if urgent",
                "Split into multiple shorter sessions",
                f"Try different dates - analyzed {len(daily_conflicts)} days with conflicts"
            ]
        }
        
        # Still try to generate some conflict details if we have a best day
        if best_resolution:
            state.conflict_resolution.update({
                "negotiation_strategy": best_resolution.get("negotiation_strategy", ""),
                "reschedulable_meetings": len(best_resolution.get('reschedulable_conflicts', [])),
            })
    
    return state

def handle_llm_failure(state: AgentState) -> AgentState:
    state.final_output = {"error": "Could not proceed due to LLM failure."}
    return state

def format_final_response(state: AgentState) -> AgentState:
    """Formats the final response, handling both success and conflict resolution cases."""
    logging.info("Formatting final response...")
    request = state.request_data
    proposed_slot = state.proposed_slot
    
    final_json = {"Request_id": request["Request_id"], "Datetime": request["Datetime"], "Location": request["Location"], "From": request["From"], "Attendees": [], "Subject": state.extracted_info.meeting_summary, "EmailContent": request["EmailContent"], "EventStart": None, "EventEnd": None, "Duration_mins": str(state.extracted_info.duration_minutes), "MetaData": {}}
    
    if proposed_slot:
        # SUCCESS CASE: Meeting slot found
        # Convert final UTC slot to the organizer's local time for display
        organizer_tz = pytz.timezone(USER_TIMEZONES.get(state.organizer, "UTC"))
        final_json["EventStart"] = proposed_slot.start.astimezone(organizer_tz).isoformat()
        final_json["EventEnd"] = proposed_slot.end.astimezone(organizer_tz).isoformat()

        # Include the context of pre-existing events on the scheduled day
        for attendee_state in state.attendees:
            # We need to re-fetch the events for the *final* chosen day to display them
            day_start_local = proposed_slot.start.astimezone(pytz.timezone(attendee_state.timezone)).replace(hour=0, minute=0, second=0)
            day_end_local = day_start_local + timedelta(days=1)
            final_day_events = get_calendar_events.invoke({"attendee_email": attendee_state.email, "start_time": day_start_local, "end_time": day_end_local})
            
            attendee_output = {"email": attendee_state.email, "events": []}
            for event in final_day_events:
                attendee_output["events"].append({"StartTime": event['start'].isoformat(), "EndTime": event['end'].isoformat(), "Summary": event['summary'], "NumAttendees": event.get('NumAttendees', "N/A"), "Attendees": event.get('Attendees', "N/A")})
            final_json["Attendees"].append(attendee_output)
            
        final_json["MetaData"]["status"] = "success"
        final_json["MetaData"]["scheduling_status"] = "confirmed"
        
    else:
        # CONFLICT CASE: No slot found, include conflict resolution data
        final_json["MetaData"]["status"] = "conflict_detected"
        final_json["MetaData"]["scheduling_status"] = "requires_negotiation"
        
        if state.conflict_resolution:
            final_json["MetaData"]["conflict_resolution"] = state.conflict_resolution
            
            # Add specific conflict resolution fields to the main response
            if state.conflict_resolution.get("override_recommended", False):
                final_json["MetaData"]["recommendation"] = "priority_override_suggested"
                final_json["Subject"] = f"[{state.conflict_resolution['priority_level'].upper()} PRIORITY] {state.extracted_info.meeting_summary}"
            else:
                final_json["MetaData"]["recommendation"] = "alternative_time_suggested"
        
        # Still populate attendee info for context, but from the first conflict day
        if hasattr(state, 'conflict_resolution') and state.conflict_resolution:
            for attendee_state in state.attendees:
                attendee_output = {"email": attendee_state.email, "events": [], "conflicts_detected": True}
                final_json["Attendees"].append(attendee_output)
    
    state.final_output = final_json
    return state

print("Cell 5/13: Graph nodes UPGRADED with multi-day search loop and conflict resolution.")

Cell 5/13: Graph nodes UPGRADED with multi-day search loop and conflict resolution.


In [7]:
# Cell 6: Building the LangGraph Workflow (Simplified for Multi-Day Search)

def decide_after_extraction(state: AgentState) -> str:
    """Routes to failure node if LLM fails, otherwise proceeds to search."""
    if state.extracted_info is None:
        return "handle_llm_failure"
    else:
        return "search_for_slot"

# The main graph definition
workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("initialize", initialize_state)
workflow.add_node("extract_details", extract_details)
workflow.add_node("search_for_slot", search_for_slot_in_range) # The new core logic node
workflow.add_node("format_final_response", format_final_response)
workflow.add_node("handle_llm_failure", handle_llm_failure)

# Define the flow
workflow.set_entry_point("initialize")
workflow.add_edge("initialize", "extract_details")
workflow.add_conditional_edges(
    "extract_details",
    decide_after_extraction,
    {
        "search_for_slot": "search_for_slot",
        "handle_llm_failure": "handle_llm_failure"
    }
)
workflow.add_edge("search_for_slot", "format_final_response")
workflow.add_edge("format_final_response", END)
workflow.add_edge("handle_llm_failure", END)

# Compile the final agent
memory = MemorySaver()
agent_runnable = workflow.compile(checkpointer=memory)

print("Cell 6/13: LangGraph workflow RE-ARCHITECTED for multi-day search.")

Cell 6/13: LangGraph workflow RE-ARCHITECTED for multi-day search.


In [8]:
# Cell 7: Flask Web Server Setup (Final Version for Exact Output Match)

app = Flask(__name__)
request_history = []

def your_meeting_assistant(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    This is the main function that processes an incoming meeting request.
    It uses the compiled LangGraph agent to perform all the work.
    This function still returns the full object with 'processed' and 'output'
    as required by one part of the spec.
    """
    logging.info(f"\n\n--- New Request Received: {data['Request_id']} ---")
    initial_state = {"request_data": data}
    config = {"configurable": {"thread_id": data['Request_id']}}
    
    final_state_snapshot = agent_runnable.invoke(initial_state, config=config)
    
    final_output = final_state_snapshot.get('final_output')
    
    processed_data = {}
    extracted_info_obj = final_state_snapshot.get('extracted_info')
    if extracted_info_obj:
        processed_data["extracted_info"] = extracted_info_obj.model_dump()
    else:
        processed_data["error"] = "LLM failed to extract details."

    processed_data["agent_status_message"] = final_state_snapshot.get('status_message', 'Done.')
    
    response_data = data.copy()
    response_data["processed"] = processed_data
    response_data["output"] = final_output
    
    logging.info(f"--- Request {data['Request_id']} Processed ---")
    return response_data

@app.route('/receive', methods=['POST'])
def receive():
    """
    The main API endpoint that receives scheduling requests.
    CRITICAL CHANGE: This endpoint now returns ONLY the 'output' dictionary
    to exactly match the final graded output specification.
    """
    data = request.get_json()
    if not data or "Request_id" not in data:
        return jsonify({"error": "Invalid JSON data"}), 400
    print(f"\n[Flask] Received request: {json.dumps(data, indent=2)}")
    
    # Run the full agent
    full_response = your_meeting_assistant(data)
    
    # Store the full response for our own debugging/logging
    request_history.append(full_response)
    
    # Extract only the 'output' part for the final response to the judges
    final_output_json = full_response.get('output', {"error": "Agent failed to produce an output."})
    
    print(f"\n[Flask] Sending final graded response: {json.dumps(final_output_json, indent=2)}")
    return jsonify(final_output_json)

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

print("Cell 7/13: Flask application finalized for exact output match.")

Cell 7/13: Flask application finalized for exact output match.


In [12]:
# Cell 8: Start the Flask Server

# We run the Flask app in a separate thread to keep the notebook interactive.
# The 'daemon=True' flag ensures the thread will close when the main notebook process ends.
flask_thread = Thread(target=run_flask, daemon=True)
flask_thread.start()

print("Cell 8/13: Flask server starting in a background thread...")
print("The server is now listening on http://0.0.0.0:5000/receive")

Cell 8/13: Flask server starting in a background thread...
The server is now listening on http://0.0.0.0:5000/receive
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [13]:
# Cell 9: Final Advanced Test (Multi-Day, Multi-Timezone)

import requests
import json
import time

# The sample JSON data designed to test all advanced features
final_test_data = {
    "Request_id": "final-advanced-test-next-week-001",
    "Datetime": "2025-07-18T14:00:00Z", # Request made on a Friday
    "Location": "Virtual / Global Call",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {"email": "usertwo.amd@gmail.com"},
        {"email": "userthree.amd@gmail.com"}
    ],
    "Subject": "Final Review of Q3 Roadmap",
    "EmailContent": "Hi all, we must finalize the Q3 roadmap. Please find an available 1-hour slot for all of us to meet within next week. Thanks."
}

# The URL of your running Flask server
server_url = "http://localhost:5000/receive"
headers = {"Content-Type": "application/json"}

print("--- Sending Final Advanced Test Request ---")
# Give the server a moment to start if it was just launched
time.sleep(2) 
response = requests.post(server_url, headers=headers, data=json.dumps(final_test_data))

print(f"\n--- Server Response (Status Code: {response.status_code}) ---")
if response.status_code == 200:
    # Print the final output JSON beautifully
    print(json.dumps(response.json(), indent=2))
else:
    print("An error occurred:")
    print(response.text)

--- Sending Final Advanced Test Request ---

--- Server Response (Status Code: 200) ---
{
  "Attendees": [
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "userthree.amd@gmail.com"
          ],
          "EndTime": "2025-07-21T07:30:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-20T16:00:00+05:30",
          "Summary": "Off Hours"
        },
        {
          "Attendees": [
            "userthree.amd@gmail.com",
            "team@amd.com",
            "usertwo.amd@gmail.com",
            "userone.amd@gmail.com"
          ],
          "EndTime": "2025-07-21T08:00:00+05:30",
          "NumAttendees": 4,
          "StartTime": "2025-07-21T07:30:00+05:30",
          "Summary": "Client Validation - Urgent"
        },
        {
          "Attendees": [
            "userthree.amd@gmail.com"
          ],
          "EndTime": "2025-07-21T10:00:00+05:30",
          "NumAttendees": 1,
          "StartTime"

In [14]:
# Cell 10: Comprehensive Test Suite (Final Corrected Version)

import requests
import json
import time

# --- Define all test cases ---

# Test Case 1: All attendees are expected to be available.
test_case_1 = {
    "Request_id": "TC1-BOTH-AVAILABLE-6118b54f",
    "Datetime": "2025-07-19T12:34:55",
    "Location": "IISc Banglore",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "Subject": "Goals Discussion",
    "EmailContent": "Hi Team. Let's meet next Thursday for 30 minutes and discuss about our Goals."
}

# Test Case 2: USERTHREE is busy, USERONE and USERTWO are free.
test_case_2 = {
    "Request_id": "TC2-USERTHREE-BUSY-6118b54f",
    "Datetime": "2025-07-19T12:34:55",
    "Location": "IISc Banglore",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "Subject": "Client Validation - Urgent",
    "EmailContent": "Hi Team. We’ve just received quick feedback from the client. Let’s prioritize resolving this promptly. Let’s meet Monday at 9:00 AM for 30 minutes to discuss and resolve this issue."
}

# Test Case 3: All attendees are busy.
test_case_3 = {
    "Request_id": "TC3-ALL-BUSY-6118b54f",
    "Datetime": "2025-07-19T12:34:55",
    "Location": "IISc Banglore",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "Subject": "Project Status",
    "EmailContent": "Hi Team. Let's meet on Tuesday at 11:00 A.M for an hour and discuss about our on-going Projects."
}

# Test Case 4: USERTHREE is busy (again, different scenario).
test_case_4 = {
    "Request_id": "TC4-USERTHREE-BUSY-AGAIN-6118b54f",
    "Datetime": "2025-07-19T12:34:55",
    "Location": "IISc Banglore",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "Subject": "Client Feedback",
    "EmailContent": "Hi Team. We’ve received the final feedback from the client. Let’s meet on Wednesday at 10:00 A.M. for 45 minutes."
}

all_test_cases = [test_case_1, test_case_2, test_case_3, test_case_4]
server_url = "http://localhost:5000/receive"
headers = {"Content-Type": "application/json"}

# --- Iterate through and run all test cases ---

# Give the server a moment to start up if it was just launched
time.sleep(2) 

for i, test_case in enumerate(all_test_cases, 1):
    print(f"\n\n{'='*20} Sending Test Case {i} ({test_case['Request_id']}) {'='*20}")
    
    response = requests.post(server_url, headers=headers, data=json.dumps(test_case))
    
    if response.status_code == 200:
        print(f"\n--- Final Output for Test Case {i} ---")
        response_json = response.json()
        
        # CRITICAL FIX: The response_json *is* the final output. We print it directly.
        print(json.dumps(response_json, indent=2))
        
    else:
        print(f"ERROR on Test Case {i}: Server returned status {response.status_code}")
        print(response.text)

print(f"\n\n{'='*20} COMPREHENSIVE TEST SUITE COMPLETE {'='*20}")



==================== Sending Test Case 1 (TC1-BOTH-AVAILABLE-6118b54f) ====================

--- Final Output for Test Case 1 ---
{
  "Attendees": [
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "userthree.amd@gmail.com"
          ],
          "EndTime": "2025-07-24T07:30:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-23T16:00:00+05:30",
          "Summary": "Off Hours"
        },
        {
          "Attendees": [
            "userthree.amd@gmail.com",
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com"
          ],
          "EndTime": "2025-07-24T11:00:00+05:30",
          "NumAttendees": 3,
          "StartTime": "2025-07-24T10:30:00+05:30",
          "Summary": "Agentic AI Project Status Update"
        },
        {
          "Attendees": [
            "userthree.amd@gmail.com"
          ],
          "EndTime": "2025-07-25T07:30:00+05:30",
          "NumAttendees": 1,

In [19]:
import requests
import json
SERVER_URL = "http://129.212.191.91"
INPUT_JSON_FILE = "JSON_Samples/Input_Request.json"
with open(INPUT_JSON_FILE) as f:
        input_json = json.load(f)
response = requests.post(SERVER_URL+":5000/receive", json=input_json, timeout=10)
print(f"\n--- Final Output for Test Case {i} ---")
response_json = response.json()

# CRITICAL FIX: The response_json *is* the final output. We print it directly.
print(json.dumps(response_json, indent=2))


--- Final Output for Test Case 4 ---
{
  "Attendees": [
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "userthree.amd@gmail.com"
          ],
          "EndTime": "2025-07-17T07:30:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-16T16:00:00+05:30",
          "Summary": "Off Hours"
        },
        {
          "Attendees": [
            "userthree.amd@gmail.com"
          ],
          "EndTime": "2025-07-18T07:30:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-17T16:00:00+05:30",
          "Summary": "Off Hours"
        }
      ]
    },
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "userone.amd@gmail.com"
          ],
          "EndTime": "2025-07-17T09:00:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-16T18:00:00+05:30",
          "Summary": "Off Hours"
        },
        {
        